In [2]:
import numpy as np
import networkx as nx
import nltk

In [5]:
# Load data


comic_graph = nx.read_gpickle('Data/GCC_uddirect.pickle.pickle')

FileNotFoundError: [Errno 2] No such file or directory: 'Data/GCC_uddirect.pickle.pickle'

# Part 1: Visualization and Statistics of the Network

## Part 1a: Basic Statistics of the Network

**What is the number of nodes in the network?**

**What is the number of links?**

**What is the number of links connecting Marvel and DC? Explain what those links mean?**

**Who are top 5 most connected characters? (Report results for in-degrees and out-degrees).**

**Plot the in and out-degree distributions.**

**Compare the degree distribution to a random network with the same number of nodes and probability of connection $p$.**